In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta
from matplotlib import pyplot as plt
import seaborn as sns
sns.set();
from tqdm import tqdm   #progressbar
tqdm.pandas()   #progressbar instantiëren voor pandas

# Hoe goed kunnen participanten hun eigen smartphonegebruik inschatten? 

per participant een verschilscore (subjectief tov objectief) verkrijgen steeds per vraag. 

In [87]:
app_filename = "D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/data/MobileDNAdata/floor_appevents.csv"
notif_filename = "D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/data/MobileDNAdata/data/floor/floor_notifications.csv"

In [88]:
df = pd.read_csv(app_filename, sep=";", parse_dates=['startTime', 'endTime'])
df.drop("Unnamed: 0", axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101531 entries, 0 to 101530
Data columns (total 14 columns):
ppn                101531 non-null int64
application        101531 non-null object
battery            101531 non-null float64
data_version       101531 non-null float64
endTime            101531 non-null datetime64[ns]
endTimeMillis      101531 non-null object
id                 101531 non-null object
latitude           101531 non-null object
longitude          101531 non-null object
model              101531 non-null object
notification       101531 non-null bool
session            101531 non-null int64
startTime          101531 non-null datetime64[ns]
startTimeMillis    101531 non-null object
dtypes: bool(1), datetime64[ns](2), float64(2), int64(2), object(7)
memory usage: 10.2+ MB


In [89]:
df_qualtrics = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/data/qualtrics/qualtricsoriginal.xlsx')

Wat betekent dit checken voor dubbels? 

In [90]:
# Checken voor dubbels
dubbels = pd.concat(g for _, g in df.groupby(["application","startTimeMillis","endTime"]) if len(g) > 1)

In [91]:
dubbels['session'].value_counts().describe()

count     682.000000
mean       10.258065
std        90.657612
min         2.000000
25%         2.000000
50%         4.000000
75%         6.000000
max      2356.000000
Name: session, dtype: float64

In [92]:
df.session.nunique()

30674

In [93]:
df.head(5)

,ppn,application,battery,data_version,endTime,endTimeMillis,id,latitude,longitude,model,notification,session,startTime,startTimeMillis
0,1,com.tinder,67.0,1.4,2018-11-06 18:31:18.495,"1,54153E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541525474,2018-11-06 18:31:16.853,"1,54153E+12"
1,1,com.tinder,67.0,1.4,2018-11-06 18:36:03.063,"1,54153E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541525645,2018-11-06 18:34:08.065,"1,54153E+12"
2,1,com.instagram.android,73.0,1.4,2018-11-07 19:24:56.992,"1,54162E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541614866,2018-11-07 19:24:54.966,"1,54162E+12"
3,1,com.android.systemui,69.0,1.4,2018-11-07 19:38:12.620,"1,54162E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541614866,2018-11-07 19:38:10.092,"1,54162E+12"
4,1,com.facebook.orca,64.0,1.4,2018-11-07 23:04:25.243,"1,54163E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541628241,2018-11-07 23:04:03.090,"1,54163E+12"


## Aantal dagen gelogd per participant

### voorbereiding

In [94]:
# Omvormen naar datetime object
df['startTime'] = pd.to_datetime(df['startTime'])
df['endTime'] = pd.to_datetime(df['endTime'])

In [95]:
# Datum variabele maken (zonder tijdstip)
df['dag'] = df.startTime.dt.date

In [96]:
Deze kolom straks gebruiken om de dagen te bepalen (week tov weekend)

SyntaxError: invalid syntax (<ipython-input-96-2b7649032df0>, line 1)

### aantal dagen gelogd, per participant

In [ ]:
# Groeperen op ppn, kijken naar totaal unieke dagen gelogd, per persoon
df["log_dag"] = df.groupby('ppn')['dag'].nunique('dag')

log_dag nog in dataframe zetten om analyse te doen. 

In [80]:
df["log_dag"].describe()

count    37.000000
mean     15.594595
std       4.579298
min       3.000000
25%      14.000000
50%      17.000000
75%      19.000000
max      19.000000
Name: log_dag, dtype: float64

## Geaggregeerd en op participantniveau

### Gemiddelde tijd gespendeerd op smartphone per dag

Deze werkt niet? met Millis

In [ ]:
#df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

In [84]:
#df['endTimeMillis'] = df['endTimeMillis'].apply(pd.to_numeric, errors='coerce')
#df['startTimeMillis'] = df['startTimeMillis'].apply(pd.to_numeric, errors='coerce')

In [97]:
df.dtypes

ppn                         int64
application                object
battery                   float64
data_version              float64
endTime            datetime64[ns]
endTimeMillis              object
id                         object
latitude                   object
longitude                  object
model                      object
notification                 bool
session                     int64
startTime          datetime64[ns]
startTimeMillis            object
dag                        object
dtype: object

In [99]:
df.head(5)

,ppn,application,battery,data_version,endTime,endTimeMillis,id,latitude,longitude,model,notification,session,startTime,startTimeMillis,dag
0,1,com.tinder,67.0,1.4,2018-11-06 18:31:18.495,"1,54153E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541525474,2018-11-06 18:31:16.853,"1,54153E+12",2018-11-06
1,1,com.tinder,67.0,1.4,2018-11-06 18:36:03.063,"1,54153E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541525645,2018-11-06 18:34:08.065,"1,54153E+12",2018-11-06
2,1,com.instagram.android,73.0,1.4,2018-11-07 19:24:56.992,"1,54162E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541614866,2018-11-07 19:24:54.966,"1,54162E+12",2018-11-07
3,1,com.android.systemui,69.0,1.4,2018-11-07 19:38:12.620,"1,54162E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541614866,2018-11-07 19:38:10.092,"1,54162E+12",2018-11-07
4,1,com.facebook.orca,64.0,1.4,2018-11-07 23:04:25.243,"1,54163E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541628241,2018-11-07 23:04:03.090,"1,54163E+12",2018-11-07


In [98]:
# Duur variabele aanmaken: eindtijd - starttijd, /1000 om in seconden
df['duur'] = (df.endTimeMillis - df.startTimeMillis)/1000

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [42]:
#totale duur, per ppn, per dag
df.groupby(["ppn","dag"]).duur.agg(['sum', 'mean'])

DataError: No numeric types to aggregate

In [37]:
# Gemiddelde tijd (sec) per dag (over dagen heen), per participant
df.groupby(['ppn','dag'])['duur'].sum().groupby('ppn').mean()

DataError: No numeric types to aggregate

In [ ]:
# Gemiddelde tijd (sec) per dag, geaggregeerd over de participanten heen
df.groupby(['ppn','dag'])['duur'].sum().mean()

### Uren dagelijks op smartphone

In [120]:
df['endTime'] = pd.to_datetime(df['endTime'])
df['dag'] = df.endTime.dt.date

df.groupby(['ppn','dag'])['duur'].sum()/3600

KeyError: 'Column not found: duur'

### Gemiddelde aantal app gebruik per dag 

In [101]:
# Gemiddeld aantal verschillende apps (nunique), per dag, per participant
df.groupby(['ppn', 'dag'])['application'].nunique().groupby('ppn').mean()

ppn
1     10.444444
2     23.500000
3     19.052632
4     18.166667
5     18.631579
6     21.210526
8     14.052632
9     16.052632
10    22.736842
11    20.363636
12    18.736842
13    21.157895
14    27.000000
15    22.888889
16    18.263158
17    18.235294
18    23.800000
19    13.176471
21    28.000000
22    14.937500
23    21.666667
24    21.210526
25    16.666667
26    13.705882
27    24.789474
28    22.052632
29    14.142857
31    12.750000
32    15.923077
33    16.764706
34    17.647059
35    17.588235
36    18.210526
38    17.368421
39    19.200000
42     5.714286
43    18.600000
Name: application, dtype: float64

In [102]:
# Gemiddeld aantal verschillende apps, per dag, geaggregeerd over de participanten
df.groupby('dag').application.nunique().mean()

173.94736842105263

### Top 10 vaakste/langste

In [103]:
# Top 10 vaakste (in aantal appevents?)
df.application.value_counts()[:10]

com.facebook.orca             11997
com.facebook.katana            9891
com.android.systemui           9517
com.instagram.android          7005
com.snapchat.android           4976
com.android.mms                4647
com.whatsapp                   4555
com.android.chrome             4433
com.spotify.music              3437
com.google.android.youtube     2248
Name: application, dtype: int64

In [104]:
# Top 10 langste (in s)
langsteApps = df.groupby(['application'])['duur']\
    .count() \
    .reset_index(name='duur') \
    .sort_values(by='duur', ascending=False)
langsteApps.reset_index()[:10]

KeyError: 'Column not found: duur'

In [ ]:
# 1. Algemeen gemiddelde tijd op gsm per dag:
# Gemiddelde tijd (sec) per dag, geaggregeerd, omgevormd naar hh:mm:ss
sec = df.groupby(['id','dag'])['duur'].sum().mean()
# sec = 5697.430628361863

N = int(sec)
min = 60
hour = 60 * 60
day = 60 * 60 * 24

DAY = N // day
HOUR = (N - (DAY)) // hour
MINUT = (N - (DAY + (HOUR * hour))) // min
SECONDS = N - (DAY + (HOUR * hour) + (MINUT * min))

print('{}:{}:{}'.format(HOUR, MINUT, SECONDS))

## Smartphone gebruik/dag ('week' tov 'weekend')

### Stap 1: welke data werd er gelogd?

In [132]:
df['dag'].value_counts()

2018-11-13    7231
2018-11-05    7118
2018-11-06    7090
2018-11-12    6906
2018-11-08    6839
2018-11-09    6656
2018-11-07    6010
2018-11-04    5824
2018-11-16    5540
2018-11-14    5357
2018-11-10    5346
2018-11-11    5331
2018-11-15    4923
2018-11-17    4725
2018-11-18    4684
2018-11-02    4065
2018-11-03    3655
2018-11-01    3049
2018-11-19    1182
Name: dag, dtype: int64

In [ ]:
WEEK
2018-11-02
2018-11-05
2018-11-06
2018-11-07
2018-11-08
2018-11-09
2018-11-12
2018-11-13
2018-11-14
2018-11-15
2018-11-16
2018-11-19
WEEKEND
2018-11-03 
2018-11-04
2018-11-10 
2018-11-11
2018-11-17 
2018-11-18

### Stap 2: data categoriseren als "WEEK" en "WEEKEND". 

In [133]:
df['day'] = df.startTime.dt.day

Dit wordt nu volledig gebasseerd op het "nummer" van de dag in november om dat deze allemaal uniek zijn. 

In [136]:
df['moment'] = np.where((df.day == 3)|(df.day == 4)|(df.day==10)|(df.day==11)|(df.day==17)|(df.day==18), 'WEEKEND', 'WEEK')

Er zijn minder dagen die behoren tot het weekend dan tot de week; dus door de dagen van het weekend te specifiëren, is de "overschot" dan dagen in de week. 

In [137]:
df.head(5)

,ppn,application,battery,data_version,endTime,endTimeMillis,id,latitude,longitude,model,notification,session,startTime,startTimeMillis,dag,uur,locatie,day,moment
0,1,com.tinder,67.0,1.4,2018-11-06 18:31:18.495,"1,54153E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541525474,2018-11-06 18:31:16.853,"1,54153E+12",2018-11-06,18,thuis,6,WEEK
1,1,com.tinder,67.0,1.4,2018-11-06 18:36:03.063,"1,54153E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541525645,2018-11-06 18:34:08.065,"1,54153E+12",2018-11-06,18,thuis,6,WEEK
2,1,com.instagram.android,73.0,1.4,2018-11-07 19:24:56.992,"1,54162E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541614866,2018-11-07 19:24:54.966,"1,54162E+12",2018-11-07,19,thuis,7,WEEK
3,1,com.android.systemui,69.0,1.4,2018-11-07 19:38:12.620,"1,54162E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541614866,2018-11-07 19:38:10.092,"1,54162E+12",2018-11-07,19,thuis,7,WEEK
4,1,com.facebook.orca,64.0,1.4,2018-11-07 23:04:25.243,"1,54163E+12",8a2f0420-8011-4adc-b128-95a40786d765,0.0,0.0,HUAWEI NMO-L31,False,1541628241,2018-11-07 23:04:03.090,"1,54163E+12",2018-11-07,23,thuis,7,WEEK


### Welke apps meestal Thuis (gezien als na 8u en voor 19u (gokje))

In [141]:

df['uur'] = df.endTime.dt.hour
df['locatie'] = np.where((df.uur > 8) & (df.uur < 19), 'school', 'thuis')
df2 = pd.DataFrame(df[['ppn','session','application','locatie','uur']])
appsThuis = df2[df2.locatie == 'thuis']

df3 = pd.DataFrame(appsThuis.groupby('application')['session'].count())
df3.sort_values('session', ascending=False)[:30]

,session
application,
com.facebook.orca,4644
com.android.systemui,3902
com.facebook.katana,3856
com.instagram.android,3013
com.snapchat.android,2349
com.android.mms,2021
com.whatsapp,1877
com.android.chrome,1787
com.spotify.music,1093


### Welke apps meestal "op school" (tussen 8 en 19u)

In [142]:
df['uur'] = df.endTime.dt.hour
df['locatie'] = np.where((df.uur < 8) & (df.uur > 19), 'school', 'thuis')
df2 = pd.DataFrame(df[['ppn','session','application','locatie','uur']])
appsSchool = df2[df2.locatie == 'school']

df3 = pd.DataFrame(appsThuis.groupby('application')['session'].count())
df3.sort_values('session', ascending=False)[:30]

,session
application,
com.facebook.orca,4644
com.android.systemui,3902
com.facebook.katana,3856
com.instagram.android,3013
com.snapchat.android,2349
com.android.mms,2021
com.whatsapp,1877
com.android.chrome,1787
com.spotify.music,1093


## Piekgebruik

## appgebruik